# Evaluate 2D MENT-Flow model

In [ ]:
import os
import pickle
import sys
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
import proplot as pplt
import skimage.filters
import torch
from ipywidgets import interact
from ipywidgets import widgets

import mentflow as mf
import experiments.load

import plotting

In [ ]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["figure.facecolor"] = "white"
pplt.rc["grid"] = False

## Load data

In [ ]:
data_name = "swissroll"
timestamp = 231202174438
script_name = "train"
datadir = f"./data_output/{data_name}/{script_name}/{timestamp}/"

In [ ]:
device = torch.device("cpu")
precision = torch.float32

In [ ]:
def grab(x):
    return x.detach().cpu().numpy()

In [ ]:
def cvt(x):
    return x.type(precision).to(device)

In [ ]:
run = experiments.load.load_run(datadir)

args = run["args"]
cfg = run["cfg"]
dist = run["dist"]
model = run["model"]
checkpoints = run["checkpoints"]
history = run["history"]

## Scalara history

In [ ]:
keys_sorted = sorted(list(history.keys()))
imax = len(history[keys_sorted[0]])

@interact(
    key=widgets.Dropdown(options=keys_sorted, value="C_norm"),
    irange=widgets.IntRangeSlider(min=0, max=imax, value=(0, imax)),
    log=False,
)
def update(key, irange, log):
    vals = history[key]
    
    avg = vals[0]
    avgs = []
    for i, val in enumerate(vals):
        avg = 0.99 * avg + 0.01 * val
        avgs.append(avg)

    fig, ax = pplt.subplots()
    ax.plot(vals[irange[0] : irange[1]], color="gray")
    ax.plot(avgs[irange[0] : irange[1]], color="black", lw=1.0)    
    if log:
        ax.format(yscale="log", yformatter="log")
    ax.format(xlabel="Iteration (global)", ylabel=key)
    plt.show()

## Evaluation

### Density

In [ ]:
@interact(
    index=widgets.IntSlider(min=0, max=(len(checkpoints) - 1), value=0),
    n=widgets.FloatLogSlider(min=2, max=6, value=1.00e+05),
    xmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    bins=widgets.IntSlider(min=4, max=150, value=125),
    res=widgets.IntSlider(min=4, max=250, value=150),
    cmap=["viridis", "plasma", "blues", "mono", pplt.Colormap("dark_r", space="hpl")],
)
def update(index, n, xmax, bins, res, cmap):
    model.load(checkpoints[index]["path"], device=device)

    n = int(n)

    x = model.sample(n)
    x = grab(x)

    x0 = dist.sample(n)

    limits = 2 * [(-xmax, xmax)]
    coords = 2 * [torch.linspace(-xmax, xmax, res)]
    x_grid = torch.vstack([C.ravel() for C in torch.meshgrid(*coords, indexing="ij")]).T
    x_grid = x_grid.to(device)
    log_prob = model.log_prob(x_grid)
    prob = torch.exp(log_prob)
    prob = prob.reshape((res, res))
    prob = grab(prob)

    fig, axs = pplt.subplots(ncols=3)
    kws = dict(cmap=cmap)
    axs[0].hist2d(x0[:, 0], x0[:, 1], bins=bins, range=limits, **kws)
    axs[1].hist2d(x[:, 0], x[:, 1], bins=bins, range=limits, **kws)
    axs[2].pcolormesh(coords[0], coords[1], prob.T, **kws)

### Projections

In [ ]:
@interact(
    index=widgets.IntSlider(min=0, max=(len(checkpoints) - 1), value=0),
    n=widgets.FloatLogSlider(min=2, max=6, value=50000),
    log_ymin=widgets.FloatSlider(min=-5.0, max=-1.0, value=-3.0),
)
def update(index, n, log_ymin, kde=True, log=False):
    model.load(checkpoints[index]["path"], device=device)

    with torch.no_grad():

        x = model.sample(int(n))
        x = cvt(x)
        predictions = model.simulate(x, kde=kde)
        
        _predictions = [grab(prediction) for prediction in predictions]
        _measurements = [grab(measurement) for measurement in model.measurements]
        
        edges = grab(model.diagnostic.bin_edges)
        coords = 0.5 * edges[:-1] + edges[1:]

        kws = dict()

        maxcols = 7
        ncols = min(len(_measurements), maxcols)
        nrows = int(np.ceil(len(_measurements) / ncols))
        figheight = 1.3 * nrows
        figwidth = min(10.0, 1.75 * ncols)
        
        fig, axs = pplt.subplots(ncols=ncols, nrows=nrows, figheight=figheight, figwidth=figwidth)
        for j in range(len(model.measurements)):
            ax = axs[j]            
            scale = np.max(_measurements[j])
            ax.plot(coords, _measurements[j] / scale, color="black", **kws)
            ax.plot(coords, _predictions[j] / scale, color="red", **kws)
        axs.format(ymax=1.25)
        axs.format(ymin=(10.0 ** log_ymin))
        if log:
            axs.format(yscale="log")
        plt.show()